In [1]:
import json
import os
import requests
from dotenv import load_dotenv
from time import time
# quitar los warnigngs de las peticiones
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
# leer el archivo .env
load_dotenv()

True

In [3]:
USER = os.getenv("USER")
PASSWORD = os.getenv("PASSWORD")

In [4]:
def get_token(url_nifi_api: str, access_payload: dict,verify=False) -> str:
    """
    Retrieves a JWT token by authenticating the user, makes
    use of the REST API `/access/token`.

    :param url_nifi_api: the basic URL to the NiFi API.
    :param access_payload: dictionary with keys 'username' & 'password' and
                           fitting values.
    :param verify: whether to verify the SSL certificate.
    :return: JWT Token
    """

    header = {
        "Accept-Encoding": "gzip, deflate, br",
        "Content-Type": "application/x-www-form-urlencoded",
        "Accept": "*/*",
    }
    response = requests.post(
        url_nifi_api + "access/token", headers=header, data=access_payload, verify = verify
    )
    return response.content.decode("ascii")

In [5]:
def get_processor(url_nifi_api: str, processor_id: str, token=None, verify=False):
    """
    Gets and returns a single processor.

    Makes use of the REST API `/processors/{processor_id}`.

    :param url_nifi_api: String
    :param processor_id: String
    :param token: JWT access token
    :param verify: whether to verify the SSL certificate.
    :returns: JSON object processor
    """

    # Authorization header
    header = {
        "Content-Type": "application/json",
        "Authorization": "Bearer {}".format(token),
    }

    # GET processor and parse to JSON
    response = requests.get(url_nifi_api + f"processors/{processor_id}", headers=header, verify=verify)
    return json.loads(response.content)

In [6]:
def update_processor_status(processor_id: str, new_state: str, token, url_nifi_api,verify=False):
    """Starts or stops a processor by retrieving the processor to get
    the current revision and finally putting a JSON with the desired
    state towards the API.

    :param processor_id: Id of the processor to receive the new state.
    :param new_state: String representing the new state, acceptable
                      values are: STOPPED or RUNNING.
    :param token: a JWT access token for NiFi.
    :param url_nifi_api: URL to the NiFi API
    :param verify: whether to verify the SSL certificate.
    :return: None
    """

    # Retrieve processor from `/processors/{processor_id}`
    processor = get_processor(url_nifi_api, processor_id, token)

    # Create a JSON with the new state and the processor's revision
    put_dict = {
        "revision": processor["revision"],
        "state": new_state,
        "disconnectedNodeAcknowledged": True,
    }

    # Dump JSON and POST processor
    payload = json.dumps(put_dict).encode("utf8")
    header = {
        "Content-Type": "application/json",
        "Authorization": "Bearer {}".format(token),
    }
    response = requests.put(
        url_nifi_api + f"processors/{processor_id}/run-status",
        headers=header,
        data=payload,
        verify=verify
    )
    return response

In [7]:
def get_processor_state(url_nifi_api: str, processor_id: str, token=None, verify=False):
    """
    Gets and returns a single processor state.

    Makes use of the REST API 'processors/{processor_id}/state'.

    :param url_nifi_api: String
    :param processor_id: String
    :param token: JWT access token
    :param verify: whether to verify the SSL certificate.
    :returns: JSON object processor's state
    """

    # Authorization header
    if token is None:
        header = {"Content-Type": "application/json"}
    else:
        header = {
            "Content-Type": "application/json",
            "Authorization": "Bearer {}".format(token),
        }

    # GET processor and parse to JSON
    response = requests.get(
        url_nifi_api + f"processors/{processor_id}/state", headers=header, verify=verify
    )
    return json.loads(response.content)

In [8]:
def clear_processor_state(url_nifi_api: str, processor_id: str, token, verify=False):
    """
    Clears the state of a processor using the REST API.

    Makes use of the REST API 'processors/{processor_id}/state/clear-requests'.

    :param url_nifi_api: String - Base URL to NiFi API
    :param processor_id: String - ID of the processor
    :param token: JWT access token
    :param verify: whether to verify the SSL certificate.
    :returns: Response object
    """
    header = {
        "Content-Type": "application/json",
        "Authorization": "Bearer {}".format(token),
    }

    # POST to clear state
    response = requests.post(
        url_nifi_api + f"processors/{processor_id}/state/clear-requests", 
        headers=header, 
        verify=verify
    )
    return response

In [9]:
def parse_state(json_obj, state_key: str):
    """
    Retrieves the value of a state by the key of the state out of the JSON.

    :param json_obj: the processor's general state.
    :param state_key: the key for the specific state.
    :raises ValueError: if the passed key cannot be found in the processor state.
    :return: value of the matching key.
    """
    states = json_obj["componentState"]["localState"]["state"]
    for state in states:
        if state["key"] == state_key:
            value = state["value"]
            return value
    raise ValueError(f"Could not find {state_key} ")

In [10]:
def pause(secs):
    init_time = time()
    while time() < init_time + secs:
        pass

In [11]:
def get_counters(url_nifi_api: str, token=None, verify=False):
    """
    Obtiene todos los contadores de NiFi usando la API REST.

    Utiliza la API REST `/counters`.

    :param url_nifi_api: URL base de la API de NiFi (ej: "http://localhost:8080/nifi-api/")
    :param token: Token de acceso JWT (opcional si no hay autenticación)
    :param verify: si se debe verificar el certificado SSL
    :returns: Objeto JSON con todos los contadores
    """
    
    header = {
            "Content-Type": "application/json",
            "Authorization": "Bearer {}".format(token),
        }

    # Solicita los contadores y los convierte a JSON
    response = requests.get(
        url_nifi_api + "counters", 
        headers=header, 
        verify=verify
    )
    
    if response.status_code == 200:
        return json.loads(response.content)
    else:
        raise Exception(f"Error al obtener contadores: {response.status_code} - {response.text}")

def get_counter_value_by_id(counters_response: dict, counter_id: str):
    """
    Obtiene el valor de un contador específico por su ID desde la respuesta JSON.

    :param counters_response: Respuesta JSON completa de la API de contadores
    :param counter_id: ID del contador a buscar
    :returns: Valor del contador o None si no se encuentra
    """
    
    # Verificar si tiene aggregateSnapshot (estructura más común)
    if "aggregateSnapshot" in counters_response["counters"]:
        for counter in counters_response["counters"]["aggregateSnapshot"]["counters"]:
            if counter["id"] == counter_id:
                return counter["value"]
    
    # Verificar si tiene nodeSnapshots (estructura alternativa)
    elif "nodeSnapshots" in counters_response["counters"]:
        for node_snapshot in counters_response["counters"]["nodeSnapshots"]:
            for counter in node_snapshot["snapshot"]["counters"]:
                if counter["id"] == counter_id:
                    return counter["value"]
    
    return None

def reset_counter(url_nifi_api: str, counter_id: str, token, verify=False):
    """
    Resetea un contador específico a 0.

    :param url_nifi_api: URL base de la API de NiFi
    :param counter_id: ID del contador a resetear
    :param token: Token de acceso JWT
    :param verify: si se debe verificar el certificado SSL
    :returns: Respuesta de la API
    """
    
    header = {
        "Content-Type": "application/json",
        "Authorization": "Bearer {}".format(token),
    }
    
    reset_payload = {
        "counter": {
            "value": 0
        }
    }
    
    response = requests.put(
        url_nifi_api + f"counters/{counter_id}",
        headers=header,
        data=json.dumps(reset_payload),
        verify=verify
    )
    
    return response

In [12]:
access_payload = {
        "username": USER,
        "password": PASSWORD,
    }  # e.g. retrieve via Airflow's `BaseHook` functionality
url_nifi_api = "https://192.9.200.15:8443/nifi-api/"  # e.g. retrieve via Airflow's `BaseHook` functionality

In [13]:
token = get_token(url_nifi_api, access_payload, verify=False)

In [44]:
todos_contadores = get_counters(url_nifi_api, token, verify=False)
todos_contadores

{'counters': {'aggregateSnapshot': {'generated': '12:10:46 CLT',
   'counters': [{'id': '40746cd7-3424-3f4e-b4fe-74082e33e77b',
     'context': 'UPSERT Oracle CramerLA_Test roles (179705e5-0318-36ed-1029-bad9441fc827)',
     'name': 'Batches Executed',
     'valueCount': 3567,
     'value': '3,567'},
    {'id': '88d3ae31-06c5-38bf-bebc-569ac091b2e3',
     'context': 'UPSERT Oracle CramerLA_Test roles (179705e5-0318-36ed-1029-bad9441fc827)',
     'name': 'UPSERT updates performed',
     'valueCount': 3567,
     'value': '3,567'},
    {'id': '85e46133-c9cd-32bc-8a1e-43432342c375',
     'context': 'Variable count_split_roles_peru (c84c6509-236b-3144-1aee-cb1146300e11)',
     'name': 'count_split_roles_peru',
     'valueCount': 165,
     'value': '165'},
    {'id': 'b2331b39-c0a1-398e-b528-22bf85c30ba3',
     'context': 'Variable count_split_overtime (539828ec-33a0-30a9-23e8-de0f11f4c682)',
     'name': 'count_split_overtime',
     'valueCount': 892,
     'value': '892'},
    {'id': 'b0fd5

In [39]:
procesador = '19cea901-ddc2-3ce6-c087-9d6e0e5bf4d2'

In [43]:
run_once = update_processor_status(procesador, "RUN_ONCE", token, url_nifi_api)

In [41]:
run_once.status_code, run_once.text

(200,
 '{"revision":{"clientId":"f9fce487-4797-475d-a999-c0b34c5f17fc","version":5,"lastModifier":"davila"},"id":"19cea901-ddc2-3ce6-c087-9d6e0e5bf4d2","uri":"https://192.9.200.15:8443/nifi-api/processors/19cea901-ddc2-3ce6-c087-9d6e0e5bf4d2","position":{"x":-2376.0,"y":-896.0},"permissions":{"canRead":true,"canWrite":true},"bulletins":[],"component":{"id":"19cea901-ddc2-3ce6-c087-9d6e0e5bf4d2","versionedComponentId":"75776492-0198-1000-3557-5bb259aa2ac1","parentGroupId":"e46f129c-ecce-31c2-8104-6003b551a8f2","position":{"x":-2376.0,"y":-896.0},"name":"Flujo Inicial overtime","type":"org.apache.nifi.processors.standard.GenerateFlowFile","bundle":{"group":"org.apache.nifi","artifact":"nifi-standard-nar","version":"2.4.0"},"state":"STOPPED","style":{},"relationships":[{"name":"success","description":"","autoTerminate":false,"retry":false}],"supportsParallelProcessing":true,"supportsBatching":true,"supportsSensitiveDynamicProperties":false,"persistsState":false,"restricted":false,"depreca